In [61]:
#coding:utf-8
import pandas as pd
from pandas import ExcelWriter
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
from pyhunter import PyHunter
import pygsheets

In [62]:
#hunter =PyHunter('26efc74b0216d1530baa4d7ae3aa69c1b843b804')
gc = pygsheets.authorize(service_file='AccathonDataBase-8d64c56e6f04.json')
sh = gc.open(' ver. Final Accathon capital - High Potential Opportunities ') 

application = sh.worksheet_by_title('Test_output_aingel output')
application= application.get_as_df(has_header=True)
table=application

#print(table)
#table['Company']
#print(table)

#names=table['Company']
#URL=table['LinkedIn']
names=table['Company']
URL=table['LinkedIn']

In [63]:
client = requests.Session()
HOMEPAGE_URL = 'https://www.linkedin.com'
LOGIN_URL = 'https://www.linkedin.com/uas/login-submit'
#LOGIN_URL = 'https://www.linkedin.com/uas/connect/user-signin's
html = client.get(HOMEPAGE_URL).content
soup = BeautifulSoup(html, "html.parser")
csrf = soup.find(id="loginCsrfParam-login")['value']
login_information = {
    'session_key':'mik279@nyu.edu',  #mik279@nyu.edu
    'session_password':'Press@VCKing729',  
    'loginCsrfParam': csrf,
}

#print(csrf)
#
LOGIN_URL = 'https://www.linkedin.com/uas/login-submit?loginSubmitSource=GUEST_HOME'
#
client.post(LOGIN_URL, data=login_information)
headers={'Accept-Encoding':'identity'}

In [64]:
############################### AUTO graphing relationships ############################
def search(string,z,position):
    df_initial = pd.DataFrame(columns=['Company Name','Person Name','Position','Industry','Past Organizations','Title','Location','Start Date','End Date','Summary','LinkedIn URL'])
    if(string==""):
        print('LinkedIn Profile unavailable for this VC')
        #occupations.append(' ')  # N/A
        #summaryofprofile.append(' ') # N/A
        #names.append(' ') # N/A
    if(string!=''):
        #if 'linkedin' in str(linkedIn_COO[z]):
        try:
            profile_url=client.get(string.lstrip(),headers=headers)
            string.lstrip()
            body=BeautifulSoup(profile_url.content,'html.parser')
            text=body.get_text()
            summary_text=body.get_text()
            ############################### Searching for Profile Summary #############################################            start_summary= summary_text.find('{"summary":"')
            start_summary= summary_text.find('{"summary":"')
            end_summary=summary_text.find(',"industryName":"')
            temp_summaryofprofile=[]
            if(start_summary==-1 or end_summary==-1): #& end_summary==-1
                #print("No summary available for this VC",linkedIn_COO[z]) 
                temp_summaryofprofile.append(' ') #N/A
                #temp_summaryofprofile='N/A'
                #print(temp_summaryofprofile)
            elif(start_summary!=-1 & end_summary!=-1):
                if(('\n\nPreviously Mamoon worked as a Venture Capital' in summary_text[start_summary+12:end_summary-1])==False):
                    temp_summaryofprofile.append(summary_text[start_summary+12:end_summary-1])
                else:
                    temp_summaryofprofile.append(' ')
                #temp_summaryofprofile=summary_text[start_summary+12:end_summary-1]
                #print(temp_summaryofprofile)
            #summaryofprofile.append(temp_summaryofprofile)  
            else:
                temp_summaryofprofile.append(' ')
            
            ############################### Searching for First Name and Last Name ##############################
            companyindex=[]
            companyName=[]
            industries=[]
            titles=[]
            locations=[]
            startdates=[]
            enddates=[]
            person_names=[]
            #for m in re.finditer('"firstName":"',text):
            #    end_index=text[m.end():].find('",')
            #    temp_first=text[(m.end()):(m.end()+end_index)]
            #    last_index=text[m.end():].find('"lastName":"')
            #    end_index2=text[(m.end()+last_index):].find('",')
            #    temp_last=text[(m.end()+last_index+12):m.end()+last_index+end_index2]
            #    temp_name=temp_first+' '+temp_last
            #    person_names.append(temp_name)
            #person_names=[x for x in person_names if x!= 'Huiyu(Joey) Zhang']
            #if person_names==[]:
            #    person_names.append('NA')
            #name_final=max(person_names,key=person_names.count)
            
            
            ################################# Education Information ###########################################
            schoolindex=[]
            schoolnames=[]
            degreenames=[]
            majors=[]
            school_startyears=[]
            for m in re.finditer('"schoolName":"',text):
                schoolindex.append(m.end())
                end_index=text[m.end():].find('",')
                temp_school=text[m.end():(m.end()+end_index)]
                schoolnames.append(temp_school)
            schoolnames=','.join(list(set(schoolnames)))

            for m in re.finditer('"degreeName":"',text):
                end_index=text[m.end():].find('",')
                temp_degree=text[m.end():(m.end()+end_index)]
                degreenames.append(temp_degree)
            degreenames=','.join(list(set(degreenames)))

            for m in re.finditer('"fieldOfStudy":"',text):
                end_index=text[m.end():].find('"')
                temp_degree=text[m.end():(m.end()+end_index)]
                majors.append(temp_degree)
            majors=','.join(list(set(majors)))

            for i in schoolindex:
                temp_text=text[(i-300):i]
                if(temp_text.find('"startDate":{"year":')!=-1):
                    startdate_start=temp_text.find('"startDate":{"year":')
                    school_startyears.append(temp_text[(startdate_start+20):(startdate_start+24)])
            school_startyears=list(map(int,school_startyears))
            if(len(school_startyears)!=0):
                min_startyear=min(school_startyears)
                roughage=(2018-min_startyear)+18
            else:
                roughage='NA'
                    
            ############################## Searching for Past Organizations ################################
            for m in re.finditer('"companyName":"',text):
                companyindex.append(m.end())
                temp_text=text[m.end():]
                second_index=m.end()+temp_text.find('","timePeriod"')
                companyName.append(text[m.end():second_index])
            companyindex.append(len(text))

            for i in range(0,(len(companyindex)-1)):
                temp_text=text[companyindex[i]:companyindex[i+1]]
                ############# Industry ###############
                if(temp_text.find('"industries":["')==-1):
                    industries.append('NA')
                else:
                    startindex=temp_text.find('"industries":["')
                    endindex=temp_text[startindex:].find('"],')
                    temp_industry=temp_text[startindex+15:(startindex+endindex)]
                    industries.append(temp_industry)
                ############### title ##################    
                if(temp_text.find('"title":"')==-1):
                    titles.append('NA')
                else:
                    startindex=temp_text.find('"title":"')
                    endindex=temp_text[startindex:].find('",')
                    temp_title=temp_text[startindex+9:(startindex+endindex)]
                    titles.append(temp_title)
                ################ location #################
                if(temp_text.find('"locationName":"')==-1):
                    locations.append('NA')
                else:
                    startindex=temp_text.find('"locationName":"')
                    endindex=temp_text[startindex:].find('",')
                    temp_location=temp_text[startindex+16:(startindex+endindex)]
                    locations.append(temp_location)
                ################ timeperiod #################
                #if(temp_text.find('"startDate":{"month":')==-1):
                #    startdates.append('NA')
                #    enddates.append('NA')
                #    startindex=temp_text.find()
                #else:
                #    startindex=temp_text.find('"startDate":{"month":')
                #    endindex=temp_text[startindex:].find(',"')
                #    temp_month=temp_text[startindex+21:(startindex+endindex)]
                #    temp_year=temp_text[(startindex+endindex+8):(startindex+endindex+12)]
                #    startdates.append(temp_month+'/'+temp_year)
                #    if(temp_text[:startindex].find('endDate":{"month":')==-1):
                #        enddates.append('NA')
                #    elif(temp_text[:startindex].find('endDate":{"month":')!=-1):
                #        temptemp_text=temp_text[:startindex]
                #        index1=temptemp_text.find('"endDate":{"month":')
                #        index2=temptemp_text[index1:].find(',"')
                #        temp_month2=temptemp_text[(index1+19):(index1+index2)]
                #        temp_year2=temptemp_text[(index1+index2+8):(index1+index2+12)]
                #        enddates.append(temp_month2+'/'+temp_year2)   

                
                ################ timeperiod #################
                if(temp_text.find('"endDate":{"month":')==-1):
                    #startdates.append('NA')
                    #enddates.append('NA')
                    if(temp_text.find('"endDate":{"year":')!=-1):
                        startindex=temp_text.find('"endDate":{"year":')
                        endindex=temp_text[startindex:].find(',"')
                        temp_year=temp_text[(startindex+18):(startindex+endindex)]
                        enddates.append(temp_year)
                        timeperiodindex=temp_text[startindex:].find('"timePeriod":')
                        if(timeperiodindex==-1):
                            if(temp_text.find('"startDate":{"month":')!=-1):
                                startindex1=temp_text.find('"startDate":{"month":')
                                endindex1=temp_text[startindex1:].find(',"')
                                temp_month=temp_text[(startindex1+21):(startindex1+endindex1)]
                                temp_year=temp_text[(startindex1+endindex1+8):(startindex1+endindex1+12)]
                                startdates.append(temp_month+'/'+temp_year)
                            else:
                                startindex2=temp_text.find('"startDate":{"year":')
                                endindex2=temp_text[startindex2:].find(',"')
                                temp_year=temp_text[(startindex2+20):(startindex2+endindex2)]
                                startdates.append(temp_year)  
                        else:
                            temptemp_text=temp_text[startindex:(startindex+timeperiodindex)]
                            if(temptemp_text.find('"startDate":{"month":')!=-1):
                                startindex1=temptemp_text.find('"startDate":{"month":')
                                endindex1=temptemp_text[startindex1:].find(',"')
                                temp_month=temptemp_text[(startindex1+21):(startindex1+endindex1)]
                                temp_year=temptemp_text[(startindex1+endindex1+8):(startindex1+endindex1+12)]
                                if(len(temp_month+'/'+temp_year)<=7):
                                    startdates.append(temp_month+'/'+temp_year)
                                else:
                                    startdates.append('NA')
                            else:
                                startindex2=temptemp_text.find('"startDate":{"year":')
                                endindex2=temptemp_text[startindex2:].find(',"')
                                temp_year=temptemp_text[(startindex2+20):(startindex2+endindex2)]
                                if(len(temp_year)==4):
                                    startdates.append(temp_year) 
                                else:
                                    startdates.append('NA')
                            
                    elif(temp_text.find('"endDate":{"year":')==-1):
                        enddates.append('NA')
                        if(temp_text.find('"startDate":{"month":')!=-1):
                            startindex=temp_text.find('"startDate":{"month":')
                            endindex=temp_text[startindex:].find(',"')
                            temp_month=temp_text[(startindex+21):(startindex+endindex)]
                            temp_year=temp_text[(startindex+endindex+8):(startindex+endindex+12)]
                            if(len(temp_month+'/'+temp_year)<=7):
                                startdates.append(temp_month+'/'+temp_year)
                            else:
                                print(table['Company'][z]+'temp_month='+temp_month+',temp_year='+temp_year)
                                startdates.append('NA')
                        else:
                            startindex=temp_text.find('"startDate":{"year":')
                            endindex=temp_text[startindex:].find(',"')
                            temp_year=temp_text[(startindex+20):(startindex+endindex)]
                            if(len(temp_year)==4):
                                startdates.append(temp_year)
                            else:
                                startdates.append('NA')
                else:
                    if(temp_text.find('"startDate":{"month":')!=-1):
                        startindex=temp_text.find('"startDate":{"month":')
                        endindex=temp_text[startindex:].find(',"')
                        temp_month=temp_text[startindex+21:(startindex+endindex)]
                        temp_year=temp_text[(startindex+endindex+8):(startindex+endindex+12)]
                        startdates.append(temp_month+'/'+temp_year)
                        if(temp_text[:startindex].find('endDate":{"month":')==-1):
                            enddates.append('NA')
                        elif(temp_text[:startindex].find('endDate":{"month":')!=-1):
                            temptemp_text=temp_text[:startindex]
                            index1=temptemp_text.find('"endDate":{"month":')
                            index2=temptemp_text[index1:].find(',"')
                            temp_month2=temptemp_text[(index1+19):(index1+index2)]
                            temp_year2=temptemp_text[(index1+index2+8):(index1+index2+12)]
                            enddates.append(temp_month2+'/'+temp_year2)
                    else:
                        startindex=temp_text.find('"startDate":{"year":')
                        endindex=temp_text[startindex:].find(',"')
                        temp_year=temp_text[(startindex+20):(startindex+endindex)]
                        startdates.append(temp_year)
                        if(temp_text[:startindex].find('endDate":{"month":')==-1):
                            enddates.append('NA')
                            temptemp_text=temp_text[:startindex]
                            index1=temptemp_text.find('"endDate":{"month":')
                            index2=temptemp_text[index1:].find(',"')
                            temp_month2=temptemp_text[(index1+19):(index1+index2)]
                            temp_year2=temptemp_text[(index1+index2+8):(index1+index2+12)]
                            enddates.append(temp_month2+'/'+temp_year2)
                                     
                        
                        
                        
            df_person=pd.DataFrame({
                'Company Name':[table['Company'][z]]*(len(companyName)),
                #'Person Name':[name_final]*(len(companyName)),
                'Person Name':[table['Name'][z]]*(len(companyName)),
                'Position':[position]*(len(companyName)),
                'Industry':industries,
                'Past Organizations':companyName,
                'Title':titles,
                'Location':locations,
                'Start Date': startdates,
                'End Date':enddates,
                'Summary':temp_summaryofprofile*(len(companyName)),
                'School Names':[schoolnames]*(len(companyindex)-1),
                'Degrees':[degreenames]*(len(companyindex)-1),
                'Majors':[majors]*(len(companyindex)-1),
                'Rough Age':[roughage]*(len(companyindex)-1),
                'LinkedIn URL':[string]*(len(companyName))
                })
            df_initial=pd.concat([df_initial,df_person])
            #print(temp_summaryofprofile)          
        except:
            #print(temp_summaryofprofile)
            print('Unsuccessful scraping for '+table['Company'][z])
            #print(len(startdates))
            #print(len(enddates))
            #print(len(companyName))
    return(df_initial)

In [74]:
sample_start=2000     #2730-2735    # 5258-5263  # 674-675 # 1687-1694
sample_end=2010
df= pd.DataFrame(columns=['Company Name','Person Name','Position','Industry','Past Organizations','Title','Location',
                          'Start Date','End Date','Summary','School Names','Degrees','Majors','Rough Age','LinkedIn URL'])
for i in range(sample_start,sample_end):  #len(linkedIn_COO) #len(URL)
    Founder=search(URL[i],i,'Founder or Investor')
    #CTO=search(linkedIn_CTO[i],i,'CTO')
    #COO=search(linkedIn_COO[i],i,'COO')
    #newdf=pd.concat([CEO,CTO,COO])  
    #df=pd.concat([df,newdf])
    df=pd.concat([df,Founder])
df

,Company Name,Degrees,End Date,Industry,LinkedIn URL,Location,Majors,Past Organizations,Person Name,Position,Rough Age,School Names,Start Date,Summary,Title


In [70]:
#authorization
gc = pygsheets.authorize(service_file='AccathonDataBase-8d64c56e6f04.json')
sh=gc.open('Startup Evaluation')
relationship=sh.worksheet_by_title('Raw')
#relationship.set_dataframe(df,start="A1")
relationship= relationship.get_as_df(has_header=True)
#print(len(relationship)+2)
newstart='A'+ str(len(relationship)+2)
#relationship.set_dataframe(df,start=newstart)
sh.worksheet_by_title('Raw').set_dataframe(df,start=newstart,copy_head=False)